### This script shows how we use the datatype autodetection of Pandas to create a schema for Spark Dataframes. Subsequently we used Spark to write our files to Parquet formats.

In [1]:
import pyspark
import os
import sys
import pandas as pd
from pyspark.sql import SparkSession


os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [11]:
appointment_pd = pd.read_csv('../data/noshowappointment.csv', nrows=1000)

In [12]:
appointment_pd.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,29872499824296,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,558997776694438,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4262962299951,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,867951213174,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8841186448183,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No


In [35]:
appointment_pd.describe()

,PatientId,AppointmentID,Age,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received
count,1.000000e+03,1.000000e+03,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,1.402902e+14,5.586122e+06,37.888000,0.092000,0.224000,0.098000,0.021000,0.021000,0.431000
std,2.450629e+14,6.847001e+04,24.367709,0.289171,0.417131,0.297463,0.143456,0.150272,0.495464
min,3.183853e+08,5.217179e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.216810e+12,5.542643e+06,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.816301e+13,5.617144e+06,36.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,9.398169e+13,5.638446e+06,58.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,9.968684e+14,5.642904e+06,98.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000


In [13]:
appointment_pd.dtypes

PatientId          int64
AppointmentID      int64
Gender            object
ScheduledDay      object
AppointmentDay    object
Age                int64
Neighbourhood     object
Scholarship        int64
Hipertension       int64
Diabetes           int64
Alcoholism         int64
Handcap            int64
SMS_received       int64
No-show           object
dtype: object

In [14]:
from pyspark.sql import types

In [44]:
initial_schema = types.StructType([
    types.StructField("PatientId", types.StringType(), True),
    types.StructField("AppointmentID", types.StringType(), True),
    types.StructField("Gender", types.StringType(), True),
    types.StructField("ScheduledDay", types.StringType(), True),
    types.StructField("AppointmentDay", types.StringType(), True),  
    types.StructField("Age", types.IntegerType(), True),
    types.StructField("Neighbourhood", types.StringType(), True),
    types.StructField("Scholarship", types.IntegerType(), True),
    types.StructField("Hipertension", types.IntegerType(), True),
    types.StructField("Diabetes", types.IntegerType(), True),
    types.StructField("Alcoholism", types.IntegerType(), True),
    types.StructField("Handcap", types.IntegerType(), True),
    types.StructField("SMS_received", types.IntegerType(), True),
    types.StructField("No-show", types.StringType(), True) 
])

In [45]:
df_appointment = spark.read \
        .option("header", "true") \
        .schema(initial_schema) \
        .csv('../data/noshowappointment.csv')

df_appointment = df_appointment \
    .withColumnRenamed('Hipertension', 'Hypertension') \
    .withColumnRenamed('Handcap', 'Handicap') \
    .withColumnRenamed('No-show', 'No_show')


In [46]:
df_appointment.registerTempTable('raw_temp')

In [50]:
df_result = spark.sql("""
SELECT 
    -- date column cleaning
    LEFT(ScheduledDay,10) AS ScheduledDay,
    LEFT(AppointmentDay,10) AS AppointmentDay,

    -- Other Columns 
    PatientId,
    AppointmentID,
    Gender,
    Age,
    Neighbourhood,
    Scholarship,
    Hypertension,
    Diabetes,
    Alcoholism,
    SMS_received,
    No_show
    
FROM
    raw_temp
    --filtering age column
WHERE Age > 0
""")

In [52]:
df_result.coalesce(1).write.parquet('../capstone/cleaned_data', mode='overwrite')

In [ ]:
#uploads data to GCS via CLI
##gsutil -m cp -r capstone/ gs://dtc_data_lake_khunmi-academy-376002/capstone